### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:

# ## District Summary
# * Calculate the total number of schools
numSchools = len(school_data.index)


# * Calculate the total number of students
numStudents = len(student_data.index)

# * Calculate the total budget
# * Calculate the total budget
totalBudget = school_data['budget'].sum()

# * Calculate the average math score
totalMathScore = student_data['math_score'].sum()

averageMathScore = totalMathScore / numStudents

#
# * Calculate the average reading score
#
totalReadingScore = student_data['reading_score'].sum()

averageReadingScore = totalReadingScore / numStudents


# * Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
#
overallPassingRate = (averageMathScore + averageReadingScore) / 2

# * Calculate the percentage of students with a passing math score (70 or greater)

mathSeventy = student_data[student_data['math_score'] >= 70]
mathSeventyAmt = len(mathSeventy)

percentMathSeventy = mathSeventyAmt / numStudents * 100

# * Calculate the percentage of students with a passing reading score (70 or greater)
readSeventy = student_data[student_data['reading_score'] >= 70]
readSeventyAmt = len(readSeventy)

percentReadSeventy = readSeventyAmt / numStudents * 100


# * Create a dataframe to hold the above results
#

dfOne = {'Total Schools': [numSchools], 'Total Students': [numStudents], 'Total Budget': [totalBudget], \
         'Average Math Score': [averageMathScore], 'Average Reading Score': [averageReadingScore],
         '% Passing Math': [percentMathSeventy], '% Passing Reading': [percentReadSeventy],
         '% Overall Passing Rate': [overallPassingRate]}
# * Optional: give the displayed data cleaner formatting

dfOne = pd.DataFrame(dfOne)
dfOne.style.hide_index()

# * Optional: give the displayed data cleaner formatting

dfOne['Total Budget'] = dfOne['Total Budget'].apply(lambda x: "${:,.2f}".format(x))
dfOne['Total Students'] = dfOne['Total Students'].apply(lambda x: "{:,}".format(x))

dfOne


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:

sdc = school_data_complete.copy()
sdcg = sdc.groupby('school_name')
schoolMeans = sdcg.mean()
#print(schoolMeans)
#Average Math Score
#Average Reading Score

#print(school_data)
sdm = pd.merge(school_data, schoolMeans, on = "school_name")

#['Student ID', 'School ID_y', 'size_y', 'budget_y']
del sdm['Student ID']
del sdm['School ID_y']
del sdm['size_y']
del sdm['budget_y']

sdm=sdm.rename(columns = {'School ID_x':'School Id'})
sdm=sdm.rename(columns = {'size_x':'size'})
sdm=sdm.rename(columns = {'budget_x':'budget'})
sdm=sdm.rename(columns = {'reading_score':'avg read score'})
sdm=sdm.rename(columns = {'math_score':'avg math score'})

sdcg = pd.DataFrame(sdcg)
pd.set_option('display.max_colwidth', 0)


#Select students from large data set that scored greater than 70
studentsPassingReading = student_data[student_data['reading_score'] >= 70]
studentsPassingMath = student_data[student_data['math_score'] >= 70]


#Count of students passing reading at each school
passingReadingTable =studentsPassingReading.groupby(['school_name'])['reading_score'].count().reset_index()
passingReadingTable.rename({'reading_score':'reading_count>70'},axis=1,inplace=True)


#Count of students passing math at each school
passingMathTable = studentsPassingMath.groupby(['school_name'])['math_score'].count().reset_index()
passingMathTable.rename({'math_score':'math_count>70'},axis=1,inplace=True)


sdm = pd.merge(sdm, passingReadingTable, on = "school_name")
sdm = pd.merge(sdm, passingMathTable, on = "school_name")



sdm['% reading >70'] = sdm['reading_count>70']/sdm['size']*100
sdm['% math >70'] = sdm['math_count>70']/sdm['size']*100


sdm['% overall passing'] = (sdm['% reading >70'] + sdm['% math >70'])/2




'''


Create an overview table that summarizes key metrics about each school, including:
School Name
School Type
Total Students
Total School Budget
Per Student Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)
'''




topFiveSchools = sdm.sort_values(by=['% overall passing'],ascending=False).head(5)
#print(topFiveSchools)

topFiveSchools



,School Id,school_name,type,size,budget,avg read score,avg math score,reading_count>70,math_count>70,% reading >70,% math >70,% overall passing
6,6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,1803,1749,97.039828,94.133477,95.586652
14,14,Thomas High School,Charter,1635,1043130,83.848930,83.418349,1591,1525,97.308869,93.272171,95.290520
9,9,Pena High School,Charter,962,585858,84.044699,83.839917,923,910,95.945946,94.594595,95.270270
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1426,1371,97.138965,93.392371,95.265668
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2204,2143,96.539641,93.867718,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
bottomFiveSchools = sdm.sort_values(by=['% overall passing'],ascending=False).tail(5)
bottomFiveSchools

,School Id,school_name,type,size,budget,avg read score,avg math score,reading_count>70,math_count>70,% reading >70,% math >70,% overall passing
13,13,Ford High School,District,2739,1763916,80.746258,77.102592,2172,1871,79.299014,68.309602,73.804308
12,12,Johnson High School,District,4761,3094650,80.966394,77.072464,3867,3145,81.222432,66.057551,73.639992
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2372,1916,81.316421,65.683922,73.500171
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2381,1946,80.739234,65.988471,73.363852
11,11,Rodriguez High School,District,3999,2547363,80.744686,76.842711,3208,2654,80.220055,66.366592,73.293323


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
'''
Reading Scores by Grade
Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
'''

grade =  school_data_complete.groupby(['school_name', 'grade']).agg({'math_score': ['mean']})
grade

math_score
                                  mean
school_name           grade           
Bailey High School    10th   76.996772
                      11th   77.515588
                      12th   76.492218
                      9th    77.083676
Cabrera High School   10th   83.154506
                      11th   82.765560
                      12th   83.277487
                      9th    83.094697
Figueroa High School  10th   76.539974
                      11th   76.884344
                      12th   77.151369
                      9th    76.403037
Ford High School      10th   77.672316
                      11th   76.918058
                      12th   76.179963
                      9th    77.361345
Griffin High School   10th   84.229064
                      11th   83.842105
                      12th   83.356164
                      9th    82.044010
Hernandez High School 10th   77.337408
                      11th   77.136029
                      12th   77.186567
                      9th    77.438495
Holden High School    10th   83.429825
                      11th   85.000000
                      12th   82.855422
                      9th    83.787402
Huang High School     10th   75.908735
                      11th   76.446602
                      12th   77.225641
                      9th    77.027251
Johnson High School   10th   76.691117
                      11th   77.491653
                      12th   76.863248
                      9th    77.187857
Pena High School      10th   83.372000
                      11th   84.328125
                      12th   84.121547
                      9th    83.625455
Rodriguez High School 10th   76.612500
                      11th   76.395626
                      12th   77.690748
                      9th    76.859966
Shelton High School   10th   82.917411
                      11th   83.383495
                      12th   83.778976
                      9th    83.420755
Thomas High School    10th   83.087886
                      11th   83.498795
                      12th   83.497041
                      9th    83.590022
Wilson High School    10th   83.724422
                      11th   83.195326
                      12th   83.035794
                      9th    83.085578
Wright High School    10th   84.010288
                      11th   83.836782
                      12th   83.644986
                      9th    83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
rgrade =  school_data_complete.groupby(['school_name', 'grade']).agg({'reading_score': ['mean']})
rgrade

reading_score
                                     mean
school_name           grade              
Bailey High School    10th   80.907183   
                      11th   80.945643   
                      12th   80.912451   
                      9th    81.303155   
Cabrera High School   10th   84.253219   
                      11th   83.788382   
                      12th   84.287958   
                      9th    83.676136   
Figueroa High School  10th   81.408912   
                      11th   80.640339   
                      12th   81.384863   
                      9th    81.198598   
Ford High School      10th   81.262712   
                      11th   80.403642   
                      12th   80.662338   
                      9th    80.632653   
Griffin High School   10th   83.706897   
                      11th   84.288089   
                      12th   84.013699   
                      9th    83.369193   
Hernandez High School 10th   80.660147   
                      11th   81.396140   
                      12th   80.857143   
                      9th    80.866860   
Holden High School    10th   83.324561   
                      11th   83.815534   
                      12th   84.698795   
                      9th    83.677165   
Huang High School     10th   81.512386   
                      11th   81.417476   
                      12th   80.305983   
                      9th    81.290284   
Johnson High School   10th   80.773431   
                      11th   80.616027   
                      12th   81.227564   
                      9th    81.260714   
Pena High School      10th   83.612000   
                      11th   84.335938   
                      12th   84.591160   
                      9th    83.807273   
Rodriguez High School 10th   80.629808   
                      11th   80.864811   
                      12th   80.376426   
                      9th    80.993127   
Shelton High School   10th   83.441964   
                      11th   84.373786   
                      12th   82.781671   
                      9th    84.122642   
Thomas High School    10th   84.254157   
                      11th   83.585542   
                      12th   83.831361   
                      9th    83.728850   
Wilson High School    10th   84.021452   
                      11th   83.764608   
                      12th   84.317673   
                      9th    83.939778   
Wright High School    10th   83.812757   
                      11th   84.156322   
                      12th   84.073171   
                      9th    83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

'''
Scores by School Spending
Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)
 
'''
#print(sdm)
# add column budget/size
sdm['per student budget'] = sdm['budget']/sdm['size']
#print(sdm)

# bins 580 , 600 , 620 , 640
bins = [577, 600, 620, 640, 660]

sd4 = sdm.copy()

sdm['spending summary'] = pd.cut(sdm["per student budget"], bins, labels=['577-600', '600-620', '620-640', '640-660'])

sdm2 = sdm.copy()
sdm2 = sdm2.sort_values(by=['spending summary'],ascending=False)
sdm2


,School Id,school_name,type,size,budget,avg read score,avg math score,reading_count>70,math_count>70,% reading >70,% math >70,% overall passing,per student budget,spending summary
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2372,1916,81.316421,65.683922,73.500171,655.0,640-660
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3748,3094,80.862999,66.752967,73.807983,652.0,640-660
12,12,Johnson High School,District,4761,3094650,80.966394,77.072464,3867,3145,81.222432,66.057551,73.639992,650.0,640-660
13,13,Ford High School,District,2739,1763916,80.746258,77.102592,2172,1871,79.299014,68.309602,73.804308,644.0,640-660
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2381,1946,80.739234,65.988471,73.363852,639.0,620-640
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1426,1371,97.138965,93.392371,95.265668,625.0,620-640
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432,4077,3318,81.933280,66.680064,74.306672,628.0,620-640
11,11,Rodriguez High School,District,3999,2547363,80.744686,76.842711,3208,2654,80.220055,66.366592,73.293323,637.0,620-640
14,14,Thomas High School,Charter,1635,1043130,83.848930,83.418349,1591,1525,97.308869,93.272171,95.290520,638.0,620-640
9,9,Pena High School,Charter,962,585858,84.044699,83.839917,923,910,95.945946,94.594595,95.270270,609.0,600-620


## Scores by School Size

* Perform the same operations as above, based on school size.

In [8]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
'''
Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).
'''
bins2 =[400, 1900, 3400, 5000]
labs = ['small', 'med', 'large']

sd4['size range'] = pd.cut(sd4["size"], bins2, labels = labs)
sd4 = sd4.sort_values(by=['size range'],ascending=False)
sd4


,School Id,school_name,type,size,budget,avg read score,avg math score,reading_count>70,math_count>70,% reading >70,% math >70,% overall passing,per student budget,size range
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3748,3094,80.862999,66.752967,73.807983,652.0,large
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432,4077,3318,81.933280,66.680064,74.306672,628.0,large
11,11,Rodriguez High School,District,3999,2547363,80.744686,76.842711,3208,2654,80.220055,66.366592,73.293323,637.0,large
12,12,Johnson High School,District,4761,3094650,80.966394,77.072464,3867,3145,81.222432,66.057551,73.639992,650.0,large
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2372,1916,81.316421,65.683922,73.500171,655.0,med
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2381,1946,80.739234,65.988471,73.363852,639.0,med
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2204,2143,96.539641,93.867718,95.203679,578.0,med
13,13,Ford High School,District,2739,1763916,80.746258,77.102592,2172,1871,79.299014,68.309602,73.804308,644.0,med
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1688,1653,95.854628,93.867121,94.860875,600.0,small
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1426,1371,97.138965,93.392371,95.265668,625.0,small


## Scores by School Type

* Perform the same operations as above, based on school type.

In [9]:
'''
Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).
'''
sd5 = sd4.sort_values(by=['type'],ascending=False)
sd5


,School Id,school_name,type,size,budget,avg read score,avg math score,reading_count>70,math_count>70,% reading >70,% math >70,% overall passing,per student budget,size range
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3748,3094,80.862999,66.752967,73.807983,652.0,large
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432,4077,3318,81.933280,66.680064,74.306672,628.0,large
11,11,Rodriguez High School,District,3999,2547363,80.744686,76.842711,3208,2654,80.220055,66.366592,73.293323,637.0,large
12,12,Johnson High School,District,4761,3094650,80.966394,77.072464,3867,3145,81.222432,66.057551,73.639992,650.0,large
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2372,1916,81.316421,65.683922,73.500171,655.0,med
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2381,1946,80.739234,65.988471,73.363852,639.0,med
13,13,Ford High School,District,2739,1763916,80.746258,77.102592,2172,1871,79.299014,68.309602,73.804308,644.0,med
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2204,2143,96.539641,93.867718,95.203679,578.0,med
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1688,1653,95.854628,93.867121,94.860875,600.0,small
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1426,1371,97.138965,93.392371,95.265668,625.0,small
